In [ ]:
Amphibian Diversity: https://www.kaggle.com/saurabhshahane/learning-the-wood-for-trees

Importing Libraries

In [1]:
import numpy as np
import pandas as pd

import plotnine as pn

from sklearn.preprocessing import OneHotEncoder


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
import plotnine as pn
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV


Loading Data

In [3]:
frogs = pd.read_csv('/blue/zoo6927/samanthashablin/AI Project/data.complete5.csv')

frogs

,Species,IUCN.Red.List.Status,iucn_2cat,Order,Family,Genus,Age_at_maturity_min_y,Age_at_maturity_max_y,Body_size_mm,Longevity_max_y,...,lgm.area.95,overlap.area.95,overlap.index.95,rel.range.size.95,hdi.lower99,cur.area.99,lgm.area.99,overlap.area.99,overlap.index.99,rel.range.size.99
0,Acris blanchardi,NaN,NaN,Anura,Hylidae,Acris,1.0,1.0,38.0,1.5,...,1329,761,0.009049,-62.282167,297.200000,126651,9414,4516,0.035657,-12.453474
1,Acris crepitans,Least Concern (LC),LC,Anura,Hylidae,Acris,1.0,1.0,38.0,4.9,...,1063,582,0.015326,-34.723424,340.200012,65142,9065,3854,0.059163,-6.186100
2,Acris gryllus,Least Concern (LC),LC,Anura,Hylidae,Acris,1.0,1.0,33.0,1.0,...,7350,582,0.020122,-2.935238,399.200000,39456,16228,2398,0.060777,-1.431353
3,Ambystoma barbouri,Near Threatened (NT),nonLC,Caudata,Ambystomatidae,Ambystoma,NaN,NaN,168.0,NaN,...,0,0,0.000000,NaN,349.000000,20940,4917,0,0.000000,-3.258694
4,Ambystoma jeffersonianum,Least Concern (LC),LC,Caudata,Ambystomatidae,Ambystoma,2.0,3.0,210.0,3.0,...,89,0,0.000000,-377.662921,385.800000,50382,4128,0,0.000000,-11.204942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Rhyacotriton variegatus,Least Concern (LC),LC,Caudata,Rhyacotritonidae,Rhyacotriton,1.5,1.5,115.0,NaN,...,2025,733,0.225886,-0.602469,796.800000,3764,2427,903,0.239904,-0.550886
98,Spea bombifrons,Least Concern (LC),LC,Anura,Scaphiopodidae,Spea,2.0,2.0,64.0,13.0,...,48434,24308,0.238927,-1.100549,181.399994,183003,268983,70851,0.387158,0.469828
99,Spea multiplicata,Least Concern (LC),LC,Anura,Scaphiopodidae,Spea,2.0,3.0,65.0,13.0,...,56991,34359,0.471530,-0.278570,200.000000,139093,121359,78335,0.563184,-0.146128
100,Taricha rivularis,Least Concern (LC),LC,Caudata,Salamandridae,Taricha,4.0,10.0,194.0,30.0,...,1938,437,0.423860,0.879728,779.799988,1161,2347,568,0.489233,1.021533


Checking Data

In [4]:
x = frogs[['Family']] #checking the very minimum to make sure data loading properly
y = frogs.iucn_2cat

print(x,y)

               Family
0             Hylidae
1             Hylidae
2             Hylidae
3      Ambystomatidae
4      Ambystomatidae
..                ...
97   Rhyacotritonidae
98     Scaphiopodidae
99     Scaphiopodidae
100     Salamandridae
101     Salamandridae

[102 rows x 1 columns] 0        NaN
1         LC
2         LC
3      nonLC
4         LC
       ...  
97        LC
98        LC
99        LC
100       LC
101       LC
Name: iucn_2cat, Length: 102, dtype: object


In [5]:
frogs['Order'] = pd.Categorical(frogs.Order)
frogs['Development'] = pd.Categorical(frogs.Development)
frogs['dessication_avoidance_reported'] = pd.Categorical(frogs.dessication_avoidance_reported)
frogs['Breeding_habitat'] = pd.Categorical(frogs.Breeding_habitat)
frogs['Larval_period_4cat'] = pd.Categorical(frogs.Larval_period_4cat)
frogs['iucn_2cat'] = pd.Categorical(frogs.iucn_2cat)
frogs

,Species,IUCN.Red.List.Status,iucn_2cat,Order,Family,Genus,Age_at_maturity_min_y,Age_at_maturity_max_y,Body_size_mm,Longevity_max_y,...,lgm.area.95,overlap.area.95,overlap.index.95,rel.range.size.95,hdi.lower99,cur.area.99,lgm.area.99,overlap.area.99,overlap.index.99,rel.range.size.99
0,Acris blanchardi,NaN,NaN,Anura,Hylidae,Acris,1.0,1.0,38.0,1.5,...,1329,761,0.009049,-62.282167,297.200000,126651,9414,4516,0.035657,-12.453474
1,Acris crepitans,Least Concern (LC),LC,Anura,Hylidae,Acris,1.0,1.0,38.0,4.9,...,1063,582,0.015326,-34.723424,340.200012,65142,9065,3854,0.059163,-6.186100
2,Acris gryllus,Least Concern (LC),LC,Anura,Hylidae,Acris,1.0,1.0,33.0,1.0,...,7350,582,0.020122,-2.935238,399.200000,39456,16228,2398,0.060777,-1.431353
3,Ambystoma barbouri,Near Threatened (NT),nonLC,Caudata,Ambystomatidae,Ambystoma,NaN,NaN,168.0,NaN,...,0,0,0.000000,NaN,349.000000,20940,4917,0,0.000000,-3.258694
4,Ambystoma jeffersonianum,Least Concern (LC),LC,Caudata,Ambystomatidae,Ambystoma,2.0,3.0,210.0,3.0,...,89,0,0.000000,-377.662921,385.800000,50382,4128,0,0.000000,-11.204942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Rhyacotriton variegatus,Least Concern (LC),LC,Caudata,Rhyacotritonidae,Rhyacotriton,1.5,1.5,115.0,NaN,...,2025,733,0.225886,-0.602469,796.800000,3764,2427,903,0.239904,-0.550886
98,Spea bombifrons,Least Concern (LC),LC,Anura,Scaphiopodidae,Spea,2.0,2.0,64.0,13.0,...,48434,24308,0.238927,-1.100549,181.399994,183003,268983,70851,0.387158,0.469828
99,Spea multiplicata,Least Concern (LC),LC,Anura,Scaphiopodidae,Spea,2.0,3.0,65.0,13.0,...,56991,34359,0.471530,-0.278570,200.000000,139093,121359,78335,0.563184,-0.146128
100,Taricha rivularis,Least Concern (LC),LC,Caudata,Salamandridae,Taricha,4.0,10.0,194.0,30.0,...,1938,437,0.423860,0.879728,779.799988,1161,2347,568,0.489233,1.021533


Transformations

In [6]:
frogs['averageage'] = frogs.Age_at_maturity_max_y + frogs.Age_at_maturity_min_y #averaging frog maturity age
#WHY:

In [7]:
newfrogs = pd.DataFrame(frogs, columns = ['Order', 'averageage', 'Reproductive_output_y', 'Development', 
                                     'dispersal_dist_max_m', 'Breeding_habitat', 
                  'Larval_period_4cat', 'dessication_avoidance_reported', 'Elev_Average', 'Total_Area', 'Bio12_Average', 'iucn_2cat']).dropna()
newfrogs

,Order,averageage,Reproductive_output_y,Development,dispersal_dist_max_m,Breeding_habitat,Larval_period_4cat,dessication_avoidance_reported,Elev_Average,Total_Area,Bio12_Average,iucn_2cat
1,Anura,2.0,2,larval,1300.00,permanent,short,yes,190.055102,8.820610e+05,1283.550190,LC
4,Caudata,5.0,1,larval,1610.00,ephemeral,mid,yes,329.398046,4.706962e+05,1074.088869,LC
5,Caudata,4.0,1,larval,405.00,ephemeral,short,yes,320.543499,1.977261e+06,896.608581,LC
6,Caudata,4.0,1,larval,1170.00,generalist_aquatic,long,yes,1174.946856,1.443399e+06,826.770188,LC
7,Caudata,5.0,1,larval,756.00,ephemeral,long,yes,273.212394,3.105941e+06,1091.476533,LC
8,Caudata,4.0,1,larval,125.00,ephemeral,mid,yes,192.677191,1.389182e+06,1112.569492,LC
9,Caudata,8.0,1,larval,600.00,generalist_aquatic,mid,yes,1047.876493,5.148908e+06,539.468054,LC
11,Caudata,6.0,1,larval,396.00,ephemeral,short,yes,79.315422,4.773361e+05,1373.389185,LC
12,Anura,10.0,1,larval,6000.00,ephemeral,short,no,1349.518153,2.853800e+06,657.171150,LC
13,Anura,4.0,1,larval,900.00,ephemeral,short,no,1142.981284,2.436574e+06,402.674078,LC


Assigning X Variables

In [8]:
#enc = OneHotEncoder(sparse=False)
#enc.fit(newfrogs)
#enc.transform(newfrogs)

pd.get_dummies(newfrogs)


,averageage,Reproductive_output_y,dispersal_dist_max_m,Elev_Average,Total_Area,Bio12_Average,Order_Anura,Order_Caudata,Development_direct,Development_larval,...,Breeding_habitat_permanent,Breeding_habitat_terrestrial,Larval_period_4cat_long,Larval_period_4cat_mid,Larval_period_4cat_none,Larval_period_4cat_short,dessication_avoidance_reported_no,dessication_avoidance_reported_yes,iucn_2cat_LC,iucn_2cat_nonLC
1,2.0,2,1300.00,190.055102,8.820610e+05,1283.550190,1,0,0,1,...,1,0,0,0,0,1,0,1,1,0
4,5.0,1,1610.00,329.398046,4.706962e+05,1074.088869,0,1,0,1,...,0,0,0,1,0,0,0,1,1,0
5,4.0,1,405.00,320.543499,1.977261e+06,896.608581,0,1,0,1,...,0,0,0,0,0,1,0,1,1,0
6,4.0,1,1170.00,1174.946856,1.443399e+06,826.770188,0,1,0,1,...,0,0,1,0,0,0,0,1,1,0
7,5.0,1,756.00,273.212394,3.105941e+06,1091.476533,0,1,0,1,...,0,0,1,0,0,0,0,1,1,0
8,4.0,1,125.00,192.677191,1.389182e+06,1112.569492,0,1,0,1,...,0,0,0,1,0,0,0,1,1,0
9,8.0,1,600.00,1047.876493,5.148908e+06,539.468054,0,1,0,1,...,0,0,0,1,0,0,0,1,1,0
11,6.0,1,396.00,79.315422,4.773361e+05,1373.389185,0,1,0,1,...,0,0,0,0,0,1,0,1,1,0
12,10.0,1,6000.00,1349.518153,2.853800e+06,657.171150,1,0,0,1,...,0,0,0,0,0,1,1,0,1,0
13,4.0,1,900.00,1142.981284,2.436574e+06,402.674078,1,0,0,1,...,0,0,0,0,0,1,1,0,1,0


In [9]:
#First one includes order. Second includes Genus.

def get_x_matrix(newfrogs):
    
   return pd.get_dummies(newfrogs)

x = get_x_matrix(newfrogs)

x

,averageage,Reproductive_output_y,dispersal_dist_max_m,Elev_Average,Total_Area,Bio12_Average,Order_Anura,Order_Caudata,Development_direct,Development_larval,...,Breeding_habitat_permanent,Breeding_habitat_terrestrial,Larval_period_4cat_long,Larval_period_4cat_mid,Larval_period_4cat_none,Larval_period_4cat_short,dessication_avoidance_reported_no,dessication_avoidance_reported_yes,iucn_2cat_LC,iucn_2cat_nonLC
1,2.0,2,1300.00,190.055102,8.820610e+05,1283.550190,1,0,0,1,...,1,0,0,0,0,1,0,1,1,0
4,5.0,1,1610.00,329.398046,4.706962e+05,1074.088869,0,1,0,1,...,0,0,0,1,0,0,0,1,1,0
5,4.0,1,405.00,320.543499,1.977261e+06,896.608581,0,1,0,1,...,0,0,0,0,0,1,0,1,1,0
6,4.0,1,1170.00,1174.946856,1.443399e+06,826.770188,0,1,0,1,...,0,0,1,0,0,0,0,1,1,0
7,5.0,1,756.00,273.212394,3.105941e+06,1091.476533,0,1,0,1,...,0,0,1,0,0,0,0,1,1,0
8,4.0,1,125.00,192.677191,1.389182e+06,1112.569492,0,1,0,1,...,0,0,0,1,0,0,0,1,1,0
9,8.0,1,600.00,1047.876493,5.148908e+06,539.468054,0,1,0,1,...,0,0,0,1,0,0,0,1,1,0
11,6.0,1,396.00,79.315422,4.773361e+05,1373.389185,0,1,0,1,...,0,0,0,0,0,1,0,1,1,0
12,10.0,1,6000.00,1349.518153,2.853800e+06,657.171150,1,0,0,1,...,0,0,0,0,0,1,1,0,1,0
13,4.0,1,900.00,1142.981284,2.436574e+06,402.674078,1,0,0,1,...,0,0,0,0,0,1,1,0,1,0


In [10]:
#y = pd.Categorical(newfrogs.iucn_2cat)
y = newfrogs.iucn_2cat.cat.codes
print(y)

1      0
4      0
5      0
6      0
7      0
8      0
9      0
11     0
12     0
13     0
14     0
19     0
20     0
23     0
26     0
30     0
31     0
33     0
39     0
40     0
43     0
44     0
46     0
49     0
51     1
53     0
54     0
57     0
59     0
60     0
61     0
64     0
68     0
70     0
71     1
72     1
79     0
82     0
84     0
86     0
89     0
90     0
91     0
92     1
93     0
94     1
95     0
96     0
97     0
98     0
100    0
dtype: int8


In [11]:
newfrogs

,Order,averageage,Reproductive_output_y,Development,dispersal_dist_max_m,Breeding_habitat,Larval_period_4cat,dessication_avoidance_reported,Elev_Average,Total_Area,Bio12_Average,iucn_2cat
1,Anura,2.0,2,larval,1300.00,permanent,short,yes,190.055102,8.820610e+05,1283.550190,LC
4,Caudata,5.0,1,larval,1610.00,ephemeral,mid,yes,329.398046,4.706962e+05,1074.088869,LC
5,Caudata,4.0,1,larval,405.00,ephemeral,short,yes,320.543499,1.977261e+06,896.608581,LC
6,Caudata,4.0,1,larval,1170.00,generalist_aquatic,long,yes,1174.946856,1.443399e+06,826.770188,LC
7,Caudata,5.0,1,larval,756.00,ephemeral,long,yes,273.212394,3.105941e+06,1091.476533,LC
8,Caudata,4.0,1,larval,125.00,ephemeral,mid,yes,192.677191,1.389182e+06,1112.569492,LC
9,Caudata,8.0,1,larval,600.00,generalist_aquatic,mid,yes,1047.876493,5.148908e+06,539.468054,LC
11,Caudata,6.0,1,larval,396.00,ephemeral,short,yes,79.315422,4.773361e+05,1373.389185,LC
12,Anura,10.0,1,larval,6000.00,ephemeral,short,no,1349.518153,2.853800e+06,657.171150,LC
13,Anura,4.0,1,larval,900.00,ephemeral,short,no,1142.981284,2.436574e+06,402.674078,LC


In [12]:
model = Pipeline([ #Our polynomial reg model
    ('scale', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)), #taking the square or whatever degree you use of ALL of our variables in x
    ('linreg', LinearRegression(fit_intercept=False)) #gets rid of the column of ones
]) #pipeline does the data transformation and model in one step

model.fit(x,y)


Pipeline(steps=[('scale', StandardScaler()), ('poly', PolynomialFeatures()),
                ('linreg', LinearRegression(fit_intercept=False))])

In [13]:
def bias(model, x, y): #GETTING BIAS
    y_hat = model.predict(x)
    error = np.mean((y_hat-y))
    return error

print('R^2 POLYNOMIAL FINAL MODEL:', model.score(x, y), 'BIAS POLYNOMIAL FINAL MODEL:', bias(model, x, y))

def MAE(model, x, y): #GETTING MEAN ABSOLUTE ERROR
    y_hat = model.predict(x)
    err = np.mean(abs(y_hat-y))
    return err

R^2 POLYNOMIAL FINAL MODEL: 1.0 BIAS POLYNOMIAL FINAL MODEL: -9.356026982566089e-17


In [14]:
kf = KFold(n_splits=4)
res = cross_val_score(model, x, y, cv=kf, scoring=MAE)
cv_mae = res.mean()
res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v KFold R^2:', cv_r2, 'c-v KFold MAE:', cv_mae)  #icky bias

model = LassoCV(normalize = True) #normalize = true to scale values to proper range so values aren't
#on a different scale than your original values
model.fit(x,y)

res = cross_val_score(model, x, y, cv=kf, scoring=MAE)
cv_MAE = res.mean()

res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v LASSO R^2:', cv_r2, 'c-v LASSO MAE:', cv_MAE)

res = cross_val_score(model, x, y, cv=kf, scoring=bias)
cv_bias = res.mean()

res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v LASSO R^2:', cv_r2, 'c-v LASSO bias:', cv_bias)

c-v KFold R^2: 0.5432046733529208 c-v KFold MAE: 0.06107849695661274
c-v LASSO R^2: 0.7499992219017456 c-v LASSO MAE: 0.0001810053981106446
c-v LASSO R^2: 0.7499992219017456 c-v LASSO bias: -1.1808367071588856e-06


In [21]:
model = Ridge(alpha=0.1, normalize=True)
model.fit(x, y)
print(model.coef_)

res = cross_val_score(model, x, y, cv=kf, scoring=MAE)
cv_MAE = res.mean()

res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v Ridge R^2:', cv_r2, 'c-v Ridge MAE:', cv_MAE)

res = cross_val_score(model, x, y, cv=kf, scoring=bias)
cv_bias = res.mean()

res = cross_val_score(model, x, y, cv=kf)
cv_r2 = res.mean()
print('c-v Ridge R^2:', cv_r2, 'c-v Ridge bias:', cv_bias)

[-9.52796723e-04 -2.47794918e-03 -2.88510965e-07  5.51145735e-06
 -2.65717556e-09  1.86441353e-06  6.53513193e-03 -6.53513193e-03
  2.42792335e-03 -2.42792335e-03  2.54262108e-04 -4.69051060e-03
  6.39661183e-04  2.42792335e-03  9.73540145e-04  2.94389365e-03
  2.42792335e-03 -4.99962687e-03 -4.08324982e-04  4.08324982e-04
 -4.69946358e-01  4.69946358e-01]
c-v Ridge R^2: 0.7457490152953882 c-v Ridge MAE: 0.015297218032867092
c-v Ridge R^2: 0.7457490152953882 c-v Ridge bias: -0.005140759327661883
